<a href="https://colab.research.google.com/github/AnkushJamthikar/StatisticalModeling/blob/main/(1)Univariate%26MultivariateCoxModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lifelines

     |████████████████████████████████| 2.5 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-survival: filename=scikit_survival-0.17.0-cp37-cp37m-linux_x86_64.whl size=4088323 sha256=82060060a2f0eea0ce5cef2b553ef62db400fd499128704f633279ad775237d8
  Stored in directory: /root/.cache/pip/wheels/91/c1/99/2c2faacc82ff4f89d27612b1fd905ea35aa24f6a93dd4dcc0d
Successfully built scikit-survival
     |████████████████████████████████| 348 kB 5.1 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=e18f0cddc119a6bb1099e281fe20ad54dcb015adc49ca07d92232ed1b36b9736
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma


In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

import joblib

In [ ]:
MTDD_TrainingData = pd.read_excel('/content/MTDD_TrainingDb.xlsx')
MTDD_TestingData  = pd.read_excel('/content/MTDD_TestingDb.xlsx')
MTDD_TrainingData.head()

,Row_ID,Region,Risk-Group (GroundTruth),GROUP,AGE,GENDER,HEIGHT,WEIGHT,BMI,BSA,SMOKER,DM,HTN,HLD,Hgb,Cr,COPD,FOLLOW_UP_DURATION,EF_orig,E_orig,E_A_orig,A_orig,e_prime_orig,E_e_prime_orig,TRV_orig,LAVi_orig,LVMi_orig,DEATH,MACCE_DEATH,TIME_TO_DEATH,ALL_REHOSPITALIZATION,TIME_TO_ALL_REHOSPITALIZATION,MACCE_REHOSPITALIZATION,TIME_TO_MACCE_REHOSPITALIZATION,MACE_REHOSPITALIZATION,TIME_TO_MACE_REHOSPITALIZATION,MACE_DEATH,TIME_TO_MACE_DEATH,MACE_CATEGORY,MACE_DEATH_REHOSPITALIZATION,TIME_TO_MACE_DEATH_REHOSPITALIZATION,Composite_Event,Time_To_CompositeEvent,Prospective,distance,GFR,CKD_GFR,Unnamed: 47,EF_norm,E_norm,E_A_norm,A_norm,e_prime_norm,E_e_prime_norm,TRV_norm,LAVi_norm,LVMi_norm,Risk-Group,1 probability,2 probability
0,2,1,1,pEF,56,1,177.8,73.028,23.100740,1.90,1.0,0,0,0,13.9,0.95,0.0,1208.546528,59.0,0.5,0.8,0.625000,7.0,7.1,1.80,26.2,66.56,0,0,1208.546528,1,369.568750,0,1208.546528,0,1208.546528,0,1208.546528,NaN,0,1208.546528,0,1208.546528,0,0.000000,87.164117,0,NaN,-0.097663,-1.262825,-0.583144,-0.566546,0.150325,-0.865810,-1.006134,-0.605876,-0.628468,1,0.98608,0.01392
1,3,1,1,pEF,56,1,177.8,72.665,22.985913,1.90,1.0,0,1,0,9.9,4.39,0.0,319.534722,59.0,0.7,1.1,0.636364,8.0,8.9,2.00,35.6,64.79,1,0,319.534722,0,319.534722,0,319.534722,0,319.534722,0,319.534722,NaN,0,319.534722,1,319.534722,0,0.148935,14.901388,1,NaN,-0.097663,-0.444858,-0.083468,-0.528243,0.537719,-0.652065,-0.631016,0.040995,-0.684295,1,0.96694,0.03306
2,6,1,1,pEF,28,1,167.0,75.899,27.214672,1.85,0.0,0,0,0,12.4,1.11,0.0,926.677083,70.0,0.9,2.3,0.391304,11.0,8.7,2.20,29.6,71.79,0,0,926.677083,1,27.989583,0,926.677083,0,926.677083,0,926.677083,NaN,0,926.677083,0,926.677083,0,0.000000,83.837274,0,NaN,0.943550,0.373110,1.915233,-1.354255,1.699903,-0.675815,-0.255897,-0.371901,-0.463510,1,0.99902,0.00098
3,7,1,1,pEF,22,1,158.0,68.399,27.399055,1.70,0.0,0,0,0,12.9,0.67,0.0,902.565972,58.0,0.8,2.3,0.347826,13.0,6.6,2.52,18.9,65.17,0,0,902.565972,1,12.857639,0,902.565972,0,902.565972,0,902.565972,NaN,0,902.565972,0,902.565972,0,0.000000,157.656114,0,NaN,-0.192319,-0.035874,1.915233,-1.500805,2.474692,-0.925183,0.344292,-1.108233,-0.672309,1,0.99939,0.00061
4,10,1,1,pEF,56,1,188.0,83.699,23.681247,2.10,1.0,0,1,0,9.2,1.08,0.0,243.829861,63.0,0.6,0.9,0.666667,7.0,8.6,2.87,28.0,98.62,1,0,243.829861,1,190.318750,0,243.829861,0,243.829861,0,243.829861,NaN,0,243.829861,1,243.829861,0,0.328709,75.172633,0,NaN,0.280960,-0.853841,-0.416585,-0.426102,0.150325,-0.687689,1.000750,-0.482007,0.382729,1,0.55735,0.44265


In [ ]:
MTDD_TestingData.head()

,Row_ID,Region,Risk-Group (GroundTruth),GROUP,AGE,GENDER,HEIGHT,WEIGHT,BMI,BSA,SMOKER,DM,HTN,HLD,Hgb,Cr,COPD,EF_orig,E_orig,E_A_orig,A_orig,e_prime_orig,E_e_prime_orig,TRV_orig,LAVi_orig,LVMi_orig,FOLLOW_UP_DURATION,DEATH,MACCE_DEATH,TIME_TO_DEATH,ALL_REHOSPITALIZATION,TIME_TO_ALL_REHOSPITALIZATION,MACCE_REHOSPITALIZATION,TIME_TO_MACCE_REHOSPITALIZATION,MACE_REHOSPITALIZATION,TIME_TO_MACE_REHOSPITALIZATION,MACE_DEATH,TIME_TO_MACE_DEATH,MACE_CATEGORY,MACE_DEATH_REHOSPITALIZATION,TIME_TO_MACE_DEATH_REHOSPITALIZATION,Composite_Event,Time_To_CompositeEvent,Prospective,distance,GFR,CKD_GFR,Unnamed: 47,EF_norm,E_norm,E_A_norm,A_norm,e_prime_norm,E_e_prime_norm,TRV_norm,LAVi_norm,LVMi_norm,Risk-Group,1 probability,2 probability
0,1,1,1,pEF,63,1,175.3,64.410,20.959913,1.79,0.0,0,0,0,13.3,1.24,0.0,70,0.9,1.9,0.473684,9.0,10.5,2.2,45.8,79.45,1047.533333,0,0,1047.533333,0,1047.533333,0,1047.533333,0,1047.533333,0,1047.533333,NaN,0,1047.533333,0,1047.533333,0,1.098393,62.580379,0,NaN,0.943550,0.373110,1.248999,-1.076580,0.925114,-0.462070,-0.255897,0.742918,-0.221907,1,0.96557,0.03443
1,5,1,1,pEF,47,0,158.5,62.899,25.037168,1.64,1.0,0,0,1,12.1,0.76,0.0,78,0.8,1.1,0.727273,11.0,7.3,1.7,28.0,61.63,923.497222,0,0,923.497222,1,68.820139,0,923.497222,0,923.497222,0,923.497222,NaN,0,923.497222,0,923.497222,0,0.000000,86.700599,0,NaN,1.700796,-0.035874,-0.083468,-0.221819,1.699903,-0.842061,-1.193694,-0.482007,-0.783964,1,0.99977,0.00023
2,8,1,1,pEF,50,1,180.0,58.967,18.199691,1.75,0.0,0,1,0,11.9,1.21,0.0,63,0.5,0.9,0.555556,7.0,7.3,2.0,13.4,83.05,362.465278,0,0,362.465278,0,362.465278,0,362.465278,0,362.465278,0,362.465278,NaN,0,362.465278,0,362.465278,0,0.000000,67.466428,0,NaN,0.280960,-1.262825,-0.416585,-0.800620,0.150325,-0.842061,-0.631016,-1.486721,-0.108361,1,0.99307,0.00693
3,18,1,1,pEF,49,0,167.6,70.760,25.190674,1.80,1.0,0,0,0,11.6,0.86,0.0,63,0.6,0.8,0.750000,10.0,5.9,1.7,25.8,58.68,1038.513194,0,0,1038.513194,0,1038.513194,0,1038.513194,0,1038.513194,0,1038.513194,NaN,0,1038.513194,0,1038.513194,0,0.000000,74.541106,0,NaN,0.280960,-0.853841,-0.583144,-0.145213,1.312508,-1.008306,-1.193694,-0.633402,-0.877009,1,0.99932,0.00068
4,47,1,1,pEF,57,1,182.9,122.470,36.610217,2.42,1.0,0,1,1,13.7,1.05,0.0,66,0.5,0.8,0.625000,8.0,7.1,2.7,38.1,81.00,508.342361,0,0,508.342361,0,508.342361,0,508.342361,0,508.342361,0,508.342361,NaN,0,508.342361,0,508.342361,0,0.381645,77.378071,0,NaN,0.564927,-1.262825,-0.583144,-0.566546,0.537719,-0.865810,0.681899,0.213035,-0.173019,1,0.86290,0.13710


# Functions for Confidence Intervals of C-index

In [ ]:
def score_stat_ci(
    X,
    score_fun,
    stat_fun=np.mean,
    n_bootstraps=2000,
    confidence_level=0.95,
    seed=None,
    reject_one_class_samples=True,
    ):
       
    np.random.seed(seed)
    
    scores = []
    for i in range(n_bootstraps):
        indices   = np.random.randint(0, len(X), len(X))  
        Sampled_X = X.iloc[indices,:]
        scores.append(score_fun(Sampled_X, scoring_method = 'concordance_index'))
    
    mean_score = np.mean(scores)
    sorted_scores = np.array(sorted(scores))
    alpha = (1.0 - confidence_level) / 2.0
    ci_lower = sorted_scores[int(round(alpha * len(sorted_scores)))]
    ci_upper = sorted_scores[int(round((1.0 - alpha) * len(sorted_scores)))]
    return mean_score, ci_lower, ci_upper, scores
    
    

def score_ci(X,
    score_fun,
    n_bootstraps=1000,
    confidence_level=0.95,
    seed=None,
    reject_one_class_samples=True):
       
    score = score_fun(X, scoring_method = 'concordance_index')
    
    _, ci_lower, ci_upper, scores = score_stat_ci(
    X=X,
    score_fun=score_fun,
    n_bootstraps=n_bootstraps,
    confidence_level=confidence_level,
    seed=seed,
    reject_one_class_samples=reject_one_class_samples,
    )
    
    return score, ci_lower, ci_upper, scores

# Using lifelines package

### Univariate Cox Proportional Hazard Analysis

In [ ]:
from lifelines import CoxPHFitter

In [ ]:
def function_train_CoxModel(TrainingData, Time, Event):

  trained_model     = CoxPHFitter().fit(TrainingData, Time, Event)
  c_index_training  = trained_model.score(TrainingData, scoring_method = 'concordance_index')
  model_summary     = trained_model.summary  

  return trained_model, c_index_training, model_summary

def function_test_CoxModel(Trained_CoxModel, TestingData):
  c_index               = Trained_CoxModel.score(TestingData, scoring_method = 'concordance_index')
  _, lower, upper, _    = score_ci(TestingData, Trained_CoxModel.score)
  c_index_df            = pd.DataFrame([[c_index, lower, upper]]).round(decimals = 2)  
  return c_index_df

In [ ]:
#FeaturesForUnivariateAnalysis = ["2 probability", "AGE", "GENDER"] 
FeaturesForUnivariateAnalysis = ["AGE", "GENDER", "EF_orig", "E_orig", "E_A_orig", "A_orig", "e_prime_orig", "E_e_prime_orig", "TRV_orig", "LAVi_orig", "LVMi_orig", "2 probability"]

In [ ]:
univariate_c_index      = list()
univariate_modelsummary = list()

for feature in FeaturesForUnivariateAnalysis:
  ColumnsToBeUsed   = [feature, "Composite_Event", "Time_To_CompositeEvent"]
  E                 = ColumnsToBeUsed[-2]
  T                 = ColumnsToBeUsed[-1]

  TrainingData      = MTDD_TrainingData[ColumnsToBeUsed]
  TrainingData      = TrainingData.fillna(TrainingData.median())

  TestingData       = MTDD_TestingData[ColumnsToBeUsed]
  TestingData       = TestingData.fillna(TestingData.median())

  trained_model, c_index_training, model_summary \
                    = function_train_CoxModel(TrainingData, T, E)                    
  c_index_df        = function_test_CoxModel(trained_model, TestingData)  
  
  univariate_modelsummary.append(model_summary)
  univariate_c_index.append(c_index_df)

univariate_model_summary_df = pd.concat(univariate_modelsummary)
univariate_c_index          = pd.concat(univariate_c_index)
univariate_c_index.columns  = ["C-index", "95% lowerCI", "95% upper CI"]

univariate_model_summary_df = pd.concat([univariate_c_index.reset_index(drop=True), univariate_model_summary_df.reset_index(drop=True)], axis=1)
univariate_model_summary_df.index = FeaturesForUnivariateAnalysis
univariate_model_summary_df

/usr/local/lib/python3.7/dist-packages/lifelines/utils/__init__.py:936: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/usr/local/lib/python3.7/dist-packages/lifelines/utils/__init__.py:936: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/usr/local/lib/python3.7/dist-packages/lifelines/utils/__init__.py:936: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")
/usr/local/lib/python3.7/dist-packages/lifelines/utils/__init__.py:936: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")


,C-index,95% lowerCI,95% upper CI,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
AGE,0.76,0.68,0.84,0.035825,1.036475,0.005127,0.025777,0.045874,1.026112,1.046943,6.987577,2.796746e-12,38.379388
GENDER,0.51,0.42,0.61,0.528049,1.695622,0.158018,0.218339,0.837760,1.244009,2.311184,3.341695,8.326855e-04,10.229941
EF_orig,0.54,0.41,0.67,-0.029496,0.970935,0.005537,-0.040347,-0.018644,0.960456,0.981529,-5.327364,9.964847e-08,23.258577
E_orig,0.71,0.62,0.80,1.461526,4.312536,0.234447,1.002018,1.921034,2.723773,6.828015,6.233926,4.548874e-10,31.033771
E_A_orig,0.49,0.36,0.63,0.367181,1.443660,0.117638,0.136616,0.597747,1.146387,1.818018,3.121290,1.800605e-03,9.117302
A_orig,0.60,0.49,0.73,0.924777,2.521306,0.232407,0.469267,1.380287,1.598823,3.976042,3.979125,6.916937e-05,13.819507
e_prime_orig,0.72,0.64,0.80,-0.245590,0.782243,0.037481,-0.319051,-0.172129,0.726839,0.841871,-6.552428,5.660905e-11,34.040176
E_e_prime_orig,0.79,0.72,0.86,0.050143,1.051422,0.005668,0.039034,0.061252,1.039806,1.063167,8.846745,9.010915e-19,59.944960
TRV_orig,0.67,0.57,0.76,0.630556,1.878656,0.130986,0.373828,0.887285,1.453287,2.428527,4.813912,1.480042e-06,19.365930
LAVi_orig,0.77,0.69,0.84,0.031279,1.031774,0.003828,0.023777,0.038782,1.024062,1.039544,8.171325,3.050218e-16,51.541937


In [ ]:
univariate_model_summary_df.to_excel('UnivariateCoxModel.xlsx')

In [ ]:
# Select only those predictor with p<0.05 i.e. significant
univariate_model_summary_df.loc[univariate_model_summary_df['p']<0.05]

,C-index,95% lowerCI,95% upper CI,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
AGE,0.76,0.68,0.84,0.035825,1.036475,0.005127,0.025777,0.045874,1.026112,1.046943,6.987577,2.796746e-12,38.379388
GENDER,0.51,0.42,0.61,0.528049,1.695622,0.158018,0.218339,0.837760,1.244009,2.311184,3.341695,8.326855e-04,10.229941
EF_orig,0.54,0.41,0.67,-0.029496,0.970935,0.005537,-0.040347,-0.018644,0.960456,0.981529,-5.327364,9.964847e-08,23.258577
E_orig,0.71,0.62,0.80,1.461526,4.312536,0.234447,1.002018,1.921034,2.723773,6.828015,6.233926,4.548874e-10,31.033771
E_A_orig,0.49,0.36,0.63,0.367181,1.443660,0.117638,0.136616,0.597747,1.146387,1.818018,3.121290,1.800605e-03,9.117302
A_orig,0.60,0.49,0.73,0.924777,2.521306,0.232407,0.469267,1.380287,1.598823,3.976042,3.979125,6.916937e-05,13.819507
e_prime_orig,0.72,0.64,0.80,-0.245590,0.782243,0.037481,-0.319051,-0.172129,0.726839,0.841871,-6.552428,5.660905e-11,34.040176
E_e_prime_orig,0.79,0.72,0.86,0.050143,1.051422,0.005668,0.039034,0.061252,1.039806,1.063167,8.846745,9.010915e-19,59.944960
TRV_orig,0.67,0.57,0.76,0.630556,1.878656,0.130986,0.373828,0.887285,1.453287,2.428527,4.813912,1.480042e-06,19.365930
LAVi_orig,0.77,0.69,0.84,0.031279,1.031774,0.003828,0.023777,0.038782,1.024062,1.039544,8.171325,3.050218e-16,51.541937


### Multivariate Cox Proportional Hazard Analysis

In [ ]:
FeaturesForMultivariateAnalysis = univariate_model_summary_df.index.to_list()

ColumnsToBeUsed   = FeaturesForMultivariateAnalysis + ["Composite_Event", "Time_To_CompositeEvent"]
E                 = ColumnsToBeUsed[-2]
T                 = ColumnsToBeUsed[-1]

In [ ]:
TrainingData      = MTDD_TrainingData[ColumnsToBeUsed]
TrainingData      = TrainingData.fillna(TrainingData.median())

TestingData       = MTDD_TestingData[ColumnsToBeUsed]
TestingData       = TestingData.fillna(TestingData.median())

trained_model, c_index_training, model_summary \
                  = function_train_CoxModel(TrainingData, T, E)                    
c_index_df        = function_test_CoxModel(trained_model, TestingData)  

MultiVariateCox = model_summary
MultiVariateCox.to_excel('MultiVariateCox.xlsx')
MultiVariateCox

/usr/local/lib/python3.7/dist-packages/lifelines/utils/__init__.py:936: UserWarning: DataFrame Index is not unique, defaulting to incrementing index instead.
  warnings.warn("DataFrame Index is not unique, defaulting to incrementing index instead.")


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
covariate,,,,,,,,,,
AGE,0.019561,1.019753,0.006440,0.006938,0.032184,1.006962,1.032707,3.037275,0.002387,8.710418
GENDER,0.155249,1.167949,0.182620,-0.202679,0.513177,0.816540,1.670591,0.850122,0.395257,1.339137
EF_orig,-0.018059,0.982103,0.007093,-0.031962,-0.004156,0.968544,0.995853,-2.545841,0.010901,6.519332
E_orig,-0.109925,0.895901,0.618130,-1.321437,1.101587,0.266752,3.008936,-0.177835,0.858852,0.219518
E_A_orig,0.361656,1.435705,0.237888,-0.104596,0.827908,0.900688,2.288527,1.520277,0.128441,2.960818
A_orig,0.886391,2.426358,0.483997,-0.062226,1.835008,0.939671,6.265186,1.831398,0.067041,3.898808
e_prime_orig,0.020429,1.020639,0.080654,-0.137649,0.178507,0.871404,1.195431,0.253290,0.800044,0.321848
E_e_prime_orig,0.000765,1.000766,0.019897,-0.038232,0.039763,0.962490,1.040564,0.038465,0.969317,0.044959
TRV_orig,0.066456,1.068714,0.133520,-0.195239,0.328151,0.822638,1.388399,0.497722,0.618680,0.692735
